In [337]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [338]:
def read_instancia(file_path):
    """"Função para a leitura das instâncias geradas"""
    name = file_path.split('/')[-1].replace('.dat', '')
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 1. Lendo quantidade de itens e períodos
    items, periods = map(int, lines[0].split())

    # 2. Lendo número de plantas
    num_plants = int(lines[1].strip())

    # 3. Lendo capacidades das plantas
    capacities = [int(lines[i + 2].strip()) for i in range(num_plants)]
    capacities = np.tile(capacities, (periods, 1)).T  # Repete as capacidades ao longo dos períodos (deixar na forma j, t)

    # 4. Lendo a matriz de produção (tempo de produção, tempo de setup, custo de setup, custo de produção)
    production_problem_data = []
    start_line = 2 + num_plants
    production_time = np.zeros((items, num_plants))  # Inicializar listas para armazenar separadamente os tempos e custos
    setup_time = np.zeros((items, num_plants))
    setup_cost = np.zeros((items, num_plants))
    production_cost = np.zeros((items, num_plants))
    for i in range(num_plants * items):  # Preencher as matrizes com os dados lidos
        plant = i // items  # Determina a planta
        item = i % items    # Determina o item
        # Extrair os dados de cada linha
        prod_time, set_time, set_cost, prod_cost = map(float, lines[start_line + i].split())
        production_time[item, plant] = prod_time  # Preencher as respectivas matrizes
        setup_time[item, plant] = set_time
        setup_cost[item, plant] = set_cost
        production_cost[item, plant] = prod_cost

    # 5. Lendo os custos de inventário
    inventory_costs_line = start_line + num_plants * items
    inventory_costs = list(map(float, lines[inventory_costs_line].split()))  # Lê todos os valores de inventory_costs como uma única lista
    inventory_costs = np.array(inventory_costs).reshape(num_plants, -1)  # Divide a lista de custos de inventário por planta
    inventory_costs = inventory_costs.T  # Deixa na forma (i, j)

    # 6. Lendo a matriz de demanda (12 linhas)
    demand_matrix = []
    demand_start_line = inventory_costs_line + 1
    
    # Leitura inicial das demandas
    for i in range(periods):  # Lê as linhas de demandas para os períodos
        demands = list(map(int, lines[demand_start_line + i].split()))
        demand_matrix.append(demands)
    
    # Agora vamos dividir os valores de cada linha combinada entre as plantas
    final_demand_matrix = []
    for demands in demand_matrix:
        period_demand = []
        for j in range(num_plants):
            # Divide a demanda combinada por planta, assumindo que cada planta tem o mesmo número de itens
            plant_demand = demands[j*items:(j+1)*items]
            period_demand.append(plant_demand)
        final_demand_matrix.append(period_demand)
    
    # Transpor a matriz de demanda para o formato correto (itens, plantas, períodos)
    final_demand_matrix = np.array(final_demand_matrix)
    final_demand_matrix = np.transpose(final_demand_matrix, (2, 1, 0))  # Converte para o formato (itens, plantas, períodos)

    # 7. Reading transfer costs directly from the document as a matrix
    transfer_cost_matrix = []
    transfer_cost_line = demand_start_line + periods

    # Read the matrix of transfer costs line by line
    while transfer_cost_line < len(lines):
        line = lines[transfer_cost_line].strip()
        if line:
            # Split the line into individual cost values and convert them to float
            row = [float(value) for value in line.split()]
            transfer_cost_matrix.append(row)
        transfer_cost_line += 1

    # Convert to a numpy array (optional, if you want to work with numpy for matrix operations)
    transfer_costs = np.array(transfer_cost_matrix)

    return {"instance": name,
            "items": items,
            "periods": periods,
            "num_plants": num_plants,
            "capacities": capacities,
            "production_time": production_time,
            "setup_time": setup_time,
            "setup_cost": setup_cost,  
            "production_cost": production_cost,
            "inventory_costs": inventory_costs,
            "demand_matrix": final_demand_matrix,
            "transfer_costs": transfer_costs}


# Leitura e interpretação dos dados

In [339]:
# Exploração a partir de instância simples
problem_data = read_instancia('../instancias/multi_plant_instances/AAA03_12_2_10.dat')

In [340]:
# Produtos (i)
I = np.array([_ for _ in range(problem_data['items'])])
# Plantas (j)
J = np.array([_ for _ in range(problem_data['num_plants'])])
# Períodos (t)
T = np.array([_ for _ in range(problem_data['periods'])])
# Demanda (i, j, t)
d = np.array(problem_data['demand_matrix'])
# Capacidade (j, t)
cap = np.array(problem_data['capacities'])
# Tempo de produção (i, j)
b = np.array(problem_data['production_time'])
# Tempo de setup (i, j)
f = np.array(problem_data['setup_time'])
# Custo de produção (i, j)
c = np.array(problem_data['production_cost'])
# Custo de setup (i, j)
s = np.array(problem_data['setup_cost'])
# Custo de transporte (j, k)
r = np.array(problem_data['transfer_costs'])
# Custo de estoque (i, j)
h = np.array(problem_data['inventory_costs'])

# Features da instância

## Nº de plantas

In [341]:
len(J)

2

## Nº de produtos

In [342]:
len(I)

10

## Nº de períodos

In [343]:
len(T)

12

## Nº de variáveis binárias

In [344]:
len(I) * len(J) * len(T)

240

## Custos de produção

Devo procurar um valor representativo dos custos de produção para a instância

In [345]:
custos = []
for t in T:
    custo_plantas = 0
    for j in J:
        for i in I:
            custo_producao_i_j = s[i, j] + c[i, j] * d[i, j, t]  # Tempo de setup + produção para suprir a demanda do produto i na planta j no período t
            custo_plantas += custo_producao_i_j
    custos.append(custo_plantas)

In [346]:
custos

[np.float64(15839.799999999997),
 np.float64(16780.8),
 np.float64(16944.300000000003),
 np.float64(17365.1),
 np.float64(16390.199999999997),
 np.float64(17147.3),
 np.float64(16523.5),
 np.float64(15530.599999999999),
 np.float64(16210.1),
 np.float64(17203.3),
 np.float64(15702.3),
 np.float64(16888.0)]

In [347]:
# Custo médio para todos os períodos
np.mean(custos) / (len(I) * len(J))

np.float64(827.1887499999999)

## Tempos de produção

## Custos de setup

## Tempos de setup

## Uso de capacidade

### Capacidade com setup

In [348]:
ocupacao = []
for t in T:
    for j in J:
        necessidade_planta = 0
        for i in I:
            tempo_producao_i = f[i, j] + b[i, j] * d[i, j, t]  # Tempo de setup + produção para suprir a demanda do produto i na planta j no período t
            necessidade_planta += tempo_producao_i
        utilizacao = necessidade_planta / cap[j, t]
        ocupacao.append(utilizacao)

In [349]:
# Essa é bem fácil de fazer reverse engineering. A média dos tight é 1.1 e da capacidade normal próxima de 1.0
np.mean(ocupacao)

np.float64(1.1154492342598603)

In [350]:
# Pode ser uma boa ideia adicionar percentis se forem trazer alguma informação, mas acredito que não é por esses parâmetros de geração
np.min(ocupacao)

np.float64(0.7309159005788218)

In [351]:
np.max(ocupacao)

np.float64(1.366064414768264)

### Capacidade sem setup

Essa não vai mudar tanto também, mas posso ver

In [352]:
ocupacao = []
for t in T:
    for j in J:
        necessidade_planta = 0
        for i in I:
            tempo_producao_i = b[i, j] * d[i, j, t]  # Tempo de produção para suprir a demanda do produto i na planta j no período t
            necessidade_planta += tempo_producao_i
        utilizacao = necessidade_planta / cap[j, t]
        ocupacao.append(utilizacao)

In [353]:
np.mean(ocupacao)

np.float64(0.9656503481998641)